## Set Environment

In [ ]:
import torch
from torch import nn
from transformers import BertModel, BertTokenizer


device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
class SentimentClassifier(nn.Module):

    def __init__(self, n_classes):
        super(SentimentClassifier, self).__init__()
        self.bert = BertModel.from_pretrained(PRE_TRAINED_MODEL_NAME)
        self.drop = nn.Dropout(p=0.3)
        self.out = nn.Linear(self.bert.config.hidden_size, n_classes)

    def forward(self, input_ids, attention_mask):
        _, pooled_output = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        output = self.drop(pooled_output)
        return self.out(output)

MAX_LEN = 160
class_names = ['negative', 'neutral', 'positive']
PRE_TRAINED_MODEL_NAME = 'bert-base-cased'

tokenizer = BertTokenizer.from_pretrained(PRE_TRAINED_MODEL_NAME)


model = SentimentClassifier(len(class_names))
model.load_state_dict(torch.load('best_model_state.bin'))
model = model.to(device)

In [ ]:
review_text = "I love completing my todos! Best app ever!!!"

encoded_review = tokenizer.encode_plus(
  review_text,
  max_length=MAX_LEN,
  add_special_tokens=True,
  return_token_type_ids=False,
  pad_to_max_length=True,
  return_attention_mask=True,
  return_tensors='pt',
)
input_ids = encoded_review['input_ids']
attention_mask = encoded_review['attention_mask']

In [ ]:
output = model(input_ids, attention_mask)
_, prediction = torch.max(output, dim=1)

print(f'Review text: {review_text}')
print(f'Sentiment  : {class_names[prediction]}')

In [ ]:
import sagemaker
import pandas as pd
from sagemaker import get_execution_role
from sagemaker.predictor import RealTimePredictor, json_serializer, json_deserializer

sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()
endpoint_name = 'training-pipeline-2020-06-14-11-38-43'

class Predictor(RealTimePredictor):
    def __init__(self, endpoint_name, sagemaker_session):
        super().__init__(endpoint_name, sagemaker_session=sagemaker_session, serializer=json_serializer, 
                         deserializer=json_deserializer, content_type='application/json')

bert_predictor = Predictor(endpoint_name, sagemaker_session=sagemaker_session)

In [ ]:
payload = 'We are very happy to include pipeline into the transformers repository.'

In [ ]:
results = bert_predictor.predict(payload)
print(results)